In [1]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

import optuna
import lightgbm as lgb

import pickle

In [2]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v02_undersampled_10_24M_altbinaria.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
semillas = [1149, 4836, 9443, 7483, 3492, 6287,  830, 3294, 5932,  815, 7070,
       1529, 3454, 6073, 7831, 1069, 3632,  953,  368, 3999, 9981,  906,
       6792, 1439, 8439, 9240, 8162,   32, 5785, 1013, 2747, 2702, 4807,
       831, 3985, 4003, 4758, 7955, 3976, 5436, 8634, 2511, 8627, 6888,
       4179, 1565, 9181, 6389, 5879, 1216, 8749, 8996,   99, 1481, 4120,
       8561, 7008, 5624,  271, 5204, 9169, 2537, 4070, 5458, 7383,  137,
       8414, 6327, 3119, 5659, 2907, 9390, 7055, 9770, 9614, 6769,  694,
       7854, 9925, 7895, 5259, 9598, 6352, 7884, 5592, 7017, 7374, 2208,
       7364, 1995, 7264, 2520, 3922, 7704, 2333, 1601, 2534, 6556, 9072,
       3046]

In [4]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106]  # entreno con todos los meses

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(393861, 964)

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [7]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

In [8]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Entrenamiento

Cargamos el study de optuna que optimizamos en el script anterior

In [9]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v15" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-23 07:46:11,904] Using an existing study with name 'competencia2_lgbm_v15' instead of creating a new one.


In [10]:
resultados = study.trials_dataframe()
resultados.shape

(100, 12)

In [11]:
study.best_trial.params

{'num_leaves': 71,
 'learning_rate': 0.020985267305321752,
 'min_data_in_leaf': 1116,
 'feature_fraction': 0.25030963234641773,
 'bagging_fraction': 0.5181205454680142}

Y finalmente tomamos el mejor modelo y lo entrenamos con la totalidad de los datos.

Generamos 100 modelos, uno para cada semilla.

In [12]:
version = 'v015' # UPDATE

best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor modelo {best_iter}")

for semilla in semillas:
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }

    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia2_{version}_s{semilla}_final.txt') # _final para indicar que es la version entrenada con todo el conjunto disponible.
    
    print(f'Modelo generado con semilla {semilla}: DONE')

Mejor cantidad de árboles para el mejor modelo 659
Modelo generado con semilla 1149: DONE
Modelo generado con semilla 4836: DONE
Modelo generado con semilla 9443: DONE
Modelo generado con semilla 7483: DONE
Modelo generado con semilla 3492: DONE
Modelo generado con semilla 6287: DONE
Modelo generado con semilla 830: DONE
Modelo generado con semilla 3294: DONE
Modelo generado con semilla 5932: DONE
Modelo generado con semilla 815: DONE
Modelo generado con semilla 7070: DONE
Modelo generado con semilla 1529: DONE
Modelo generado con semilla 3454: DONE
Modelo generado con semilla 6073: DONE
Modelo generado con semilla 7831: DONE
Modelo generado con semilla 1069: DONE
Modelo generado con semilla 3632: DONE
Modelo generado con semilla 953: DONE
Modelo generado con semilla 368: DONE
Modelo generado con semilla 3999: DONE
Modelo generado con semilla 9981: DONE
Modelo generado con semilla 906: DONE
Modelo generado con semilla 6792: DONE
Modelo generado con semilla 1439: DONE
Modelo generado co